In [1]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

# Define Time Period and Parameters

In [2]:
START_DATE = dt.datetime(1970,1,1)
END_DATE = dt.datetime(2020,12,31)
T = 12

# Define Asset Allocation

In [3]:
portfolios = {'GEM': ['SPY', 'VEU', 'BND'],
              'GBM': ['SPY', 'VEU', 'BND'],
             'benchmark': ['SPY'],
             'sixtyForty': ['SPY', 'BND'],
             'Permanent': ['VTI', 'BIL', 'TLT', 'GLD']}
momentum = ['GEM', 'GBM']
fixed_portfolio = ['sixtyForty']

# Get Price Data

In [4]:
combined_assets = []

for portfolio in portfolios.keys():
    combined_assets = combined_assets + portfolios[portfolio]

combined_assets = list(set(combined_assets))
combined_assets

['VEU', 'SPY', 'TLT', 'VTI', 'BND', 'GLD', 'BIL']

In [5]:
prices = pd.DataFrame()
for asset in combined_assets:
    prices[asset] = web.DataReader(asset, 'yahoo', START_DATE, END_DATE)['Adj Close']
prices.dropna(inplace=True)
prices.reset_index(inplace=True)

RemoteDataError: Unable to read URL: https://finance.yahoo.com/quote/VEU/history?period1=32400&period2=1609491599&interval=1d&frequency=1d&filter=history
Response Text:
b'<!DOCTYPE html>\n  <html lang="en-us"><head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n      <meta charset="utf-8">\n      <title>Yahoo</title>\n      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n      <style>\n  html {\n      height: 100%;\n  }\n  body {\n      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n      background-size: cover;\n      height: 100%;\n      text-align: center;\n      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;\n  }\n  table {\n      height: 100%;\n      width: 100%;\n      table-layout: fixed;\n      border-collapse: collapse;\n      border-spacing: 0;\n      border: none;\n  }\n  h1 {

## Convert Daily Prices to Monthly Prices

In [ ]:
prices['STD_YM'] = prices['Date'].map(lambda x : dt.datetime.strftime(x, '%Y-%m'))

In [ ]:
month_list = prices['STD_YM'].unique()
monthly_prices = pd.DataFrame()
for m in month_list:
    monthly_prices = monthly_prices.append(prices[prices['STD_YM'] == m].iloc[-1,:])

In [ ]:
monthly_prices = monthly_prices.drop(columns=['STD_YM'], axis=1)

In [ ]:
monthly_prices.set_index('Date', inplace=True)
monthly_prices

## 1. GEM Portfolio

In [ ]:
gem_prices = pd.DataFrame()
for col in monthly_prices.columns:
    if col in portfolios['GEM']:
        gem_prices[col] = monthly_prices[col]
gem_prices

In [ ]:
monthly_momentum = gem_prices.copy()
monthly_momentum = monthly_momentum.apply(lambda x: x.shift(1)/x.shift(12) - 1, axis=0)
monthly_momentum.dropna(inplace=True)
monthly_momentum.head(20)

In [ ]:
rank_df = monthly_momentum.rank(axis=1)
for col in rank_df.columns:
    rank_df[col] = np.where(rank_df[col] > 2, 1, 0)
rank_df

In [ ]:
monthly_gem_returns = gem_prices.pct_change()
monthly_gem_returns.dropna(inplace=True)
monthly_gem_returns = monthly_gem_returns[rank_df.index[0].strftime('%Y-%m-%d'):]
monthly_gem_returns

In [ ]:
port = np.multiply(rank_df, monthly_gem_returns)

In [ ]:
port_returns = port.sum(axis=1)
port_returns

In [ ]:
port_cum_returns = np.exp(np.log1p(port_returns).cumsum())
port_cum_returns

## 2. GBM Portfolio

In [ ]:
gbm_prices = pd.DataFrame()
for col in monthly_prices.columns:
    if col in portfolios['GBM']:
        gbm_prices[col] = monthly_prices[col]
gbm_prices

In [ ]:
gbm_momentum = gbm_prices.copy()
gbm_momentum = gbm_momentum.apply(lambda x: x.shift(1)/x.shift(12) - 1, axis=0)
gbm_momentum.dropna(inplace=True)
gbm_momentum

In [ ]:
gbm_rank = gbm_momentum.rank(axis=1)
for col in gbm_rank.columns:
    gbm_rank[col] = np.where(gbm_rank[col] > 2, 1, 0)
gbm_rank

In [ ]:
monthly_gbm_returns = gbm_prices.pct_change()
monthly_gbm_returns.dropna(inplace=True)
monthly_gbm_returns = monthly_gbm_returns[gbm_rank.index[0].strftime('%Y-%m-%d'):]
monthly_gbm_returns

In [ ]:
gbm_sixty = np.multiply(gbm_rank, monthly_gbm_returns)
gbm_sixty_returns = gbm_sixty.sum(axis=1)
gbm_sixty_returns

In [ ]:
gbm_port = pd.DataFrame()
gbm_port['GBM_sixty'] = gbm_sixty_returns
gbm_port['GBM_forty'] = monthly_gbm_returns['BND']
weight = np.array([0.6, 0.4])
gbm_port['port_return'] = gbm_port.dot(weight)
gbm_port

In [ ]:
gbm_cum_returns = (1 + gbm_port['port_return']).cumprod()
gbm_cum_returns

## 3. 60/40 Fixed Portfolio

In [ ]:
sixtyForty = pd.DataFrame()
for col in monthly_prices.columns:
    if col in portfolios['sixtyForty']:
        sixtyForty[col] = monthly_prices[col]
sixtyForty

In [ ]:
sixtyForty_returns = sixtyForty.pct_change()
sixtyForty_returns = sixtyForty_returns[rank_df.index[0].strftime('%Y-%m-%d'):]
sixtyForty_weights = np.array([0.4, 0.6])
sixtyForty_returns['port'] = sixtyForty_returns.dot(sixtyForty_weights)
sixtyForty_returns

In [ ]:
sixtyForty_cum_returns = np.exp(np.log1p(sixtyForty_returns['port']).cumsum())
sixtyForty_cum_returns

## 4. S&P 500 Index

In [ ]:
benchmark_prices = monthly_prices['SPY']

benchmark_returns = benchmark_prices.pct_change()

benchmark_returns = benchmark_returns[rank_df.index[0].strftime('%Y-%m-%d'):]

benchmark_cum_returns = np.exp(np.log1p(benchmark_returns).cumsum())
benchmark_cum_returns

## Combined Cumulative Returns of All Portfolios for Comparison

In [ ]:
combined_df = pd.DataFrame()
combined_df['GEM'] = port_cum_returns
combined_df['GBM'] = gbm_cum_returns
combined_df['Sixty Forty'] = sixtyForty_cum_returns
combined_df['benchmark'] = benchmark_cum_returns
combined_df.iloc[0] = 1
combined_df

In [ ]:
stats_summary = pd.DataFrame(columns = ['Portfolio', 'CAGR (%)', 'MDD (%)', 'CAGR/MDD'])

for col in combined_df.columns:
    # compute CAGR
    cagr = combined_df[col]**(T/len(combined_df[col].index)) - 1
    portfolio_cagr = cagr.loc[cagr.index[-1]]

    # compute MDD
    cumulative_returns = combined_df[col]
    previous_peaks = cumulative_returns.cummax()
    drawdown = (cumulative_returns - previous_peaks) / previous_peaks
    portfolio_mdd = drawdown.min()
    
    # save CAGR and MDD for each portfolio
    
    stats_summary = stats_summary.append({'Portfolio': col,
                                         'CAGR (%)': portfolio_cagr * 100,
                                         'MDD (%)': portfolio_mdd * 100,
                                         'CAGR/MDD': abs(portfolio_cagr / portfolio_mdd).round(2)}, ignore_index=True) 

In [ ]:
stats_summary.set_index('Portfolio', inplace=True)
stats_summary.sort_values('CAGR/MDD', ascending=False, inplace=True)
stats_summary

# Performance Visualization

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(combined_df)
plt.legend(combined_df.columns)
plt.xlabel('Date')
plt.ylabel('Returns')
plt.title('Portfolio Performance Comparison')